(extra:exceptions)=
# Extra: Exceções

Já mostrei vários exemplos de erros e exceções encontradas ao tentarmos realizar alguma operação não suportada. Existe um número muito grande de exceções e erros embutidos em Python. Porém, não existe uma diferença muito significativa concreta entre as exceções, é algo mais semântico, mas existem argumentos que podem ser fornecidos às excepções. Veja também estes links para a documentação oficial mencionando [erros e exceções embutidas](https://docs.python.org/3/library/exceptions.html) e o próprio [tutorial oficial de erros e exceções](https://docs.python.org/3/tutorial/errors.html).

Para lançar uma exceção, você pode utilizar a *keyword* `raise` seguido da exceção que queremos chamar.

In [1]:
def dividir(valor1, valor2):
    if valor2 == 0:
        raise ZeroDivisionError
    return valor1 / valor2


dividir(1, 0)

ZeroDivisionError: 

Para exemplificar o que quis dizer com uma exceção ser "semântica", eu posso substituir o código acima para lançar um erro diferente.

In [2]:
def dividir(valor1, valor2):
    if valor2 == 0:
        raise ConnectionAbortedError
    return valor1 / valor2


dividir(1, 0)

ConnectionAbortedError: 

Mas isso só confundirá o usuário que ver a mensagem de erro, então não faça isso.

De forma geral, se alguma função que você definir receber algum argumento inválido ou se deparar com uma situação inesperada, é melhor lançar uma exceção do que continuar a operar com algum valor errado. Por exemplo, quando definimos nossa função de `dividir`, poderíamos ter retornado simplesmente um número muito grande quando `valor2` fosse zero, imitando um "infinito", ou retornar `None`, para significar que algo de errado aconteceu. Mas ambas essas situações podem causar problemas depois quando notarmos valores esquisitos em nosso código, e teremos que caçar o ponto onde apareceram. Isso não aconteceria se simplesmente utilizássemos uma exceção para alertar o usuário logo no início.

Em uma função, podemos lançar mais de um tipo de erro, ou o mesmo tipo de erro com diferentes mensagens. Quando lançamos a exceção, também podemos fornecer um pequeno texto explicando o que  Veja este exemplo.

In [3]:
def checar_palindromo(string):
    if len(string) == 0:
        raise ValueError("O comprimento da string precisa ser maior que 0!")
    if len(string) > 100:
        raise NotImplementedError(
            "Não implementei a checagem para strings com comprimento maior que 100 caracteres"
        )
    if any(i.isnumeric() for i in string):
        raise NotImplementedError("A string não pode conter qualquer número!")

    sanitizada = string.strip().lower()
    for pontuação in ",.!?-;: ":
        sanitizada = sanitizada.replace(pontuação, "")
    reversa = sanitizada[::-1]
    return sanitizada == sanitizada[::-1]

In [4]:
checar_palindromo("")

ValueError: O comprimento da string precisa ser maior que 0!

In [5]:
checar_palindromo("a" * 101)

NotImplementedError: Não implementei a checagem para strings com comprimento maior que 100 caracteres

In [6]:
checar_palindromo("101")

NotImplementedError: A string não pode conter qualquer número!

In [7]:
checar_palindromo("Socorram-me, subi no onibus em marrocos!")

True

Podemos capturar as exceções utilizando blocos `try...except`. Tudo que está dentro de um bloco `try` é executado. Se uma exceção é encontrada, a execução passa para o bloco `except`. Podemos capturar exceções específicas com `except (tipo da exceção)`, por exemplo, `except ZeroDivisionError`, e podemos obter o objeto da exceção com `except (tipo da exceção) as (nome)`.

In [8]:
try:
    valor = checar_palindromo("101")
except NotImplementedError:
    valor = None
print(valor)

None


Nesse exemplo, capturamos um `TypeError`. Note que a variável `valor`, do exemplo anterior, não teria valor, pois a função lançou um erro. Logo, se tentarmos acessar o conteúdo de `valor` depois do block `try...except`, teríamos um `NameError`. Neste caso, decidi colocar o valor de `None` para valor, mas isso não é indicado, pois efetivamente "escondemos" o erro.

Podemos ter mais de um `except`, e serão checados em ordem. Aqui, utilizei a *keyword* `pass`, que significa *não faça nada*. Novamente, isso não é recomendado, mas estou utilizando isso somente como exemplo.

In [9]:
try:
    checar_palindromo("101")
except NotImplementedError:
    pass
except ValueError:
    pass

Apesar de termos checado 3 condições, ainda existem erros que podem ser lançados. Veja este exemplo, onde uma exceção não prevista foi lançada. É importante pensar bem nos erros que podem ocorrer e o que fazer em todas as situações.

In [10]:
try:
    checar_palindromo(101)
except NotImplementedError:
    pass
except ValueError:
    pass

TypeError: object of type 'int' has no len()

A exceção `Exception` engloba todas as outras. Podemos utilizá-la para capturar tudo, sem precisar definir o tipo de exceção. Isso também é pouco recomendado, pois especificidade ajuda muito na resolução de bugs. Note que, como as exceções são checadas de cima pra baixo, se `except Exception` estiver em cima, as outras exceções serão ignoradas.

In [11]:
try:
    checar_palindromo("101")
except Exception as e:
    print(e.args)
except NotImplementedError:
    pass
except ValueError:
    pass

('A string não pode conter qualquer número!',)


Veja que a exceção lançada foi capturada pela `Exception` genérica, que foi utilizada como o objeto chamado `e` para pegar os argumentos dessa exceção, que neste caso é uma mensagem. Note que tal string é lançada por `NotImplementedError`.

Exceções podem ser lançadas dentro de outras exceções se você cometer algum erro.

In [12]:
try:
    checar_palindromo("101")
except NotImplementedError:
    1 / 0

ZeroDivisionError: division by zero

Por fim, podemos ter um `finally` depois de um block `try...except` que sempre será executado mesmo se uma exceção for lançada.

In [13]:
try:
    checar_palindromo("101")
except NotImplementedError:
    pass
except ValueError:
    pass
finally:
    print("Um erro foi encontrado")

Um erro foi encontrado


## Exercício resolvido

### Espiar o conteúdo de um arquivo

Faça uma função que pergunte a um usuário o nome de um arquivo, que será aberto e as primeiras linhas serão exibidas na tela. Isso precisa se repetir até o usuário fornecer um nome válido. Utilize exceções para direcionar o tipo de erro que ocorreu, caso um nome inválido seja fornecido.

**Solução**:

Utilizaremos um loop `while` infinito que só será quebrado quando pudermos abrir um arquivo. Primeiro perguntaremos o nome do arquivo, tentaremos abri-lo, utilizaremos vários `except` para capturar as exceções, depois um loop `for` para imprimir *n* linhas e por fim sair do loop.

In [14]:
def espiar(num_linhas):
    while True:
        nome = input("Qual o nome do arquivo que você quer espiar?")
        try:
            handle = open(nome, "r", encoding="utf8")
        except FileNotFoundError:
            print("O arquivo não foi encontrado. Tente novamente.")
            continue
        except IsADirectoryError:
            print("Isso é uma pasta, não um arquivo. Tente novamente.")
            continue
        except PermissionError:
            print(f"Você não possui permissão para abrir {nome} como um arquivo.")
            continue

        for i in range(num_linhas):
            print(handle.readline())
        handle.close()
        break

## Exercícios extra

### Definição de suas próprias exceções

```{admonition} Pré-requisitos
* Declaração de classes e programação orientada a objetos.
```

Declare algumas exceções próprias para função de palíndromos e reimplemente-a com essas exceções.

In [ ]:
%load "Soluções de exercícios/criar_exceções.py"